In [ ]:
### Running questions
### How to handle Ns? [We're probably going to ignore them]
### Should I remove reference from alternate the same way I remove alternate from the reference? 
### Find longest string rather than checking entire stretch and neighbors to see if they're homopolymers? 

### Note: I will clean up stuff for sure (moving lines to functions, etc), this is more about logic 

In [1]:
import cyvcf2
from pyfaidx import Fasta
import os
import argparse

In [2]:
os.chdir('/Users/quinlan/u1264408/Git/spermseq')
os.listdir()

['20_shared_rare_1SCS014.vcf',
 '1SCS014.indels_removed_homopolymer.vcf',
 'YL99029.indels.vcf',
 'GRCh38_simplerepeats_1-6_homopolymers_1bpslop.bed',
 'XH9R013.indels.vcf',
 '1SCS014.reliable_indels.vcf',
 'sites_in_more_than_5_cut.bed',
 'GRCh38_simplerepeats_1-6_homopolymers.bed',
 '.git',
 'XH9R013.reliable_indels_any_overlap.vcf',
 'UA6U012.reliable_indels.vcf',
 'XH9R013.reliable_indels.vcf',
 'GRCh38_simplerepeats_1-6_homopolymers_5bpslop.bed',
 '1SCS014.indels.vcf',
 '1SCS014.bedtools.save',
 'K8J7003.indels.vcf',
 'GRCh38_simplerepeats_1-6_homopolymers_2bpslop.bed',
 'K8J7003.reliable_indels_any_overlap.vcf',
 'K8J7003.reliable_indels.vcf',
 'YL99029.reliable_indels.vcf',
 'YL99029.reliable_indels_any_overlap.vcf',
 '1SCS014.trustworthyindels.vcf',
 'UA6U012.reliable_indels_any_overlap.vcf',
 'sites_in_more_than_5.bed',
 'shared_blood_sperm_mutation_analysis.bed',
 '1SCS014.reliable_indels_any_overlap.vcf',
 'UA6U012.indels.vcf']

In [3]:
A = '1SCS014.indels.vcf'
B = 'K8J7003.indels.vcf'
C = 'UA6U012.indels.vcf'
D = 'YL99029.indels.vcf'
E = 'XH9R013.indels.vcf'

In [4]:
genome = Fasta('/Users/quinlan/u1264408/human_g1k_v38_decoy_phix.fasta', as_raw=True) # look into as_RAW
# as_raw: optional parameter to specify whether to return sequences as a Sequence() object or as a raw string.
from cyvcf2 import VCF

In [5]:
### 1 
### GOOD TO GO  
### Simplest code: is alternate allele in reference genome sequence twice the length of alternate allele

# vcf = cyvcf2.VCF(sys.argv[2])
vcf = VCF(A)

failcount = 0
passcount = 0 

for v in vcf():
    #lenalt = max(len(v.ALT[0]), len(v.REF))
    lenalt = len(v.ALT[0])
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if v.ALT[0] in genome_seq_stretch:
        print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_stretch)
        failcount += 1 
    else:
        print('PASS', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 

[W::vcf_parse] Contig 'chr1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] FILTER '-' is not defined in the header
[W::vcf_parse] INFO 'donor' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'variation_type' is not defined in the header, assuming Type=String
[W::vcf_parse] INFO 'subtype' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'filter' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'sample' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'AF' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'alt_depth' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'depth' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'no_calls' is not defined in the header, assuming Type=String
[W::vcf_parse_format] FORMAT 'tissue' is not defined in t

FAIL  chr1 75806922 T TA
TTAA
PASS chr1 75881235 C CAT
AACGTG
FAIL  chr1 100353868 G GT
TGTT
FAIL  chr1 243572906 TA T
TA
FAIL  chr1 243572907 AC A
AC
FAIL  chr1 243572908 C CT
ACTT
FAIL  chr1 243572908 CT C
CT
FAIL  chr1 243572908 CTT C
CT
FAIL  chr2 38997474 G GT
AGTT
FAIL  chr2 39007201 T TA
ATAA
FAIL  chr2 39013443 TAA T
TA
FAIL  chr2 39013443 TA T
TA
FAIL  chr2 39013443 T TA
CTAA
FAIL  chr2 39035155 CA C
CA
FAIL  chr2 39035155 C CA
GCAA
FAIL  chr2 39035331 AT A
AT
FAIL  chr2 47408384 C CT
ACTT
FAIL  chr2 47408384 CT C
CT
FAIL  chr2 47408384 CTT C
CT
FAIL  chr2 47414420 TAAA T
TA
FAIL  chr2 47414420 TAA T
TA
FAIL  chr2 47474913 T TAGG
AATTAGGA
FAIL  chr2 47475310 TTG T
TT
FAIL  chr2 47475311 TG T
TG
FAIL  chr2 47475312 G GT
TGTT
FAIL  chr2 47475312 GT G
GT
FAIL  chr2 47791154 TTG T
TT
FAIL  chr2 47791154 T TTG
GTTTGT
FAIL  chr2 47804889 CT C
CT
FAIL  chr2 47805601 A AT
GATT
FAIL  chr2 47805601 A ATT
TGATTT
FAIL  chr2 47805601 AT A
AT
FAIL  chr2 47805601 ATT A
AT
FAIL  chr2 47805734

[W::vcf_parse] Contig 'chr3' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr4' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr5' is not defined in the header. (Quick workaround: index the file with tabix.)


FAIL  chr3 50118454 TGAGA T
TG
FAIL  chr3 50118454 TGA T
TG
FAIL  chr3 50118454 T TGA
AGTGAG
FAIL  chr4 1804285 TG T
TG
FAIL  chr4 1804285 T TG
GTGG
FAIL  chr4 54731256 T TAG
GGTAGA
FAIL  chr4 54732045 A AT
GATT
FAIL  chr5 80675095 C CA
ACAA
FAIL  chr5 80864776 AAAAT A
AA
FAIL  chr5 80864777 AAATG A
AA
FAIL  chr5 112775612 TA T
TA
FAIL  chr5 112775612 T TA
TTAA
FAIL  chr5 112839514 TAAAAG T
TA
FAIL  chr5 112840921 TA T
TA
FAIL  chr5 112841744 GA G
GA
FAIL  chr5 112842691 A ATACATCTCCAGGTAGACAGATGAGCCAACAGAACCTTACCAAACAAACAGGTTTATCCAAGAATGCCAGTAGTATTCCAAGAAGTGAGTCTGCCTCCAAAGGACTAAATCAGATGAATAATGGTAATGGAGCCAATAAAAAGGTAGAACTTTCTAGAATGTCTTCAACTAAATCAAGTGGAAGTGAATCTGATAGATCAGAAAGACCTGTATTAGTACGCCAGTCAACTTTCATCAAAGAAGCTCCAAGCCCAACCTTAAGAAGAAAATTGGAGGAATCTGCTTCATTTGAATCTCTTTCTCCATCATCTAGACCAGCTTCTCCCACTAGGTCCCAGGCACAAACTCCAGTTTTAAG
CTAAAAAAGGCCCACCCCTTAAGACTCCAGCCTCCAAAAGCCCTAGTGAAGGTCAAACAGCCACCACTTCTCCTAGAGGAGCCAAGCCATCTGTGAAATCAGAATTAAGCCCTGTTGCCAGGCAGACATCCCAAATAGGTGGGTCAAGTAAAGCACCTTCTAG

[W::vcf_parse] Contig 'chr6' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr7' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr8' is not defined in the header. (Quick workaround: index the file with tabix.)


...

[W::vcf_parse] Contig 'chr14' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr15' is not defined in the header. (Quick workaround: index the file with tabix.)


FAIL  chr14 95105297 AG A
AG
FAIL  chr14 95105298 GA G
GA
FAIL  chr14 95105298 G GA
AGAA
FAIL  chr14 95132538 CT C
CT
FAIL  chr14 103966662 CT C
CT
FAIL  chr14 103998790 GC G
GC
FAIL  chr14 103998791 C CT
GCTT
FAIL  chr14 103998791 CT C
CT
FAIL  chr14 103998791 CTT C
CT
FAIL  chr14 104004361 GTTTA G
GT
FAIL  chr14 104004361 GTTTATTTA G
GT
FAIL  chr15 89273310 TA T
TA
FAIL  chr15 89273511 TAAAA T
TA
FAIL  chr15 89273511 TAAA T
TA
FAIL  chr15 89273511 TAA T
TA
FAIL  chr15 89273511 TA T
TA
FAIL  chr15 89307731 A AG
CAGG
FAIL  chr16 89792607 GT G
GT
FAIL  chr16 89795865 CA C
CA
FAIL  chr17 41847791 C CACTGCACCCGGCTGAAAAGCTTTTTTTCATGGAAGATAATGGAAGAGGGAGTTATGAGATCACTGGTACCCCCAACAAGGGCTTCCTCAGTGAACAGAATCAATGGTTAGCAGTCAACCGTGTTTCTTTTGCTATCCACAGGTAGGTGGCTTTGATGGAGCTAATCGACTTAGGAGTGCCGAAGCCTACAGCCCTGTGGCTAACACTTGGCGCACAATCCCCACTATGTTTAATCCTCGTAGCAATTTTGGCATCGAGGTGGTGGATGACCTCTTGTTTGTGGTGGGTGGCTTTAATGGTTTTACCACCACCTTTAATGTTGAGTGCTATGATGAAAAGACCGATGAGTGGTATGATGCTCATGACATGAGTATATACCGCAGTGCTCTGAGCTG

[W::vcf_parse] Contig 'chr16' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr17' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr19' is not defined in the header. (Quick workaround: index the file with tabix.)


FAIL  chr17 58611365 G GA
CGAA
FAIL  chr19 1220729 G GC
CGCC
FAIL  chr19 4097205 TA T
TA
FAIL  chr19 4110475 GCCCTGC G
GC
FAIL  chr19 4117638 AC A
AC
FAIL  chr19 14093595 AACCCTCAGCAGGCTTAAGGAGGGGAGGCCCACCTTCCTCTGGTAG A
AA
FAIL  chr19 43543340 TCGTG T
TC
FAIL  chr19 43543341 C CGT
ATCGTG
FAIL  chr19 43543341 CGT C
CG
FAIL  chr19 43543341 CGTGT C
CG
FAIL  chr19 50401797 TG T
TG
FAIL  chr19 50414747 TG T
TG
FAIL  chr19 50416412 TG T
TG
FAIL  chr19 57235121 GCA G
GC
FAIL  chr20 59864399 G GA
AGAA
FAIL  chr20 59864457 GA G
GA
FAIL  chr20 59864457 G GA
AGAA
FAIL  chr22 41784278 GT G
GT
FAIL  chr22 41793806 A AT
GATT
FAIL  chr22 41793806 AT A
AT
FAIL  chr22 41795898 T TG
TTGG
FAIL  chrX 70678891 TG T
TG
FAIL  chrX 70678892 GAAA G
GA
FAIL  chrX 70678892 GAA G
GA
FAIL  chrX 70678892 GA G
GA
FAIL  chrX 70678934 GTCTCCCTCTCCCTCTCCC G
GT
PASS chrX 70678952 C CCACGG
TCTCCCTCTCCC
FAIL  chrX 70740803 GA G
GA
FAIL  chrX 70740803 G GA
AGAA
FAIL  chrX 70744234 GA G
GA
FAIL  chrX 70744241 A AT
AAAT
FAIL

[W::vcf_parse] Contig 'chr20' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr22' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chrX' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chrY' is not defined in the header. (Quick workaround: index the file with tabix.)


In [ ]:
### 2
###  GOOD TO GO (I THINK)
### More complex: let's think about allele size between reference and alternate alleles 
## Insertions: alt allele > ref allele
## Version where there's repeat? Or I remove ref allele? 
## Look for alt allele in reference genome 
## Deletions: ref allele > alt allele 
## Is alt allele contained within reference allele? Is the string difference in the left or right stretch?

# vcf = cyvcf2.VCF(sys.argv[2])
vcf = VCF(A)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if len(v.ALT[0]) > len(v.REF) and v.ALT[0] in genome_seq_stretch:
        print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif len(v.ALT[0]) < len(v.REF):
        valt = v.ALT[0]
        vref = v.REF
        if valt in vref:
            vref = vref.replace(valt, '', 1)
            #only replace once 
            if vref in genome_seq_left or genome_seq_right:
                print('FAIL ', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                print('PASS ', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1
        else:
            print('PASS ', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        print('PASS', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
### 3
###  Probably good to go 
### Less strict: checks for larger alt alleles in genome stretch and only homopolymers fail with regards to reference allele in adjacent stretches
vcf = VCF(A)

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]

    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'no_left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'no_right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]

    if len(v.ALT[0]) > len(v.REF) and v.ALT[0] in genome_seq_stretch:
        print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (v.ALT[0] not in genome_seq_stretch):
        print('PASS should not be in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        valt = v.ALT[0]
        vref = v.REF
        if vref in valt:
            vref = vref.replace(valt, '', 1)
            if vref in genome_seq_left_use or genome_seq_right_use:
                print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
                print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1

        else:
            print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
            print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
### 4, fine but could be fiddled with 
### Least strict: only homopolymers fail with regards to reference allele in adjacent stretches AND alternate alleles in homopolymers
vcf = VCF('1SCS014.trustworthyindels.vcf')

failcount = 0
passcount = 0 

for v in vcf():
    lenalt = max(len(v.ALT[0]), len(v.REF))
    genome_seq_left = genome[v.CHROM][v.POS-lenalt:v.POS]
    if (genome_seq_left == len(genome_seq_left) * genome_seq_left[0]):
        genome_seq_left_use = genome_seq_left
    else:
        genome_seq_left_use = 'Left'
    genome_seq_right = genome[v.CHROM][v.POS:v.POS+lenalt]
    if (genome_seq_right == len(genome_seq_right) * genome_seq_right[0]):
        genome_seq_right_use = genome_seq_right
    else:
        genome_seq_right_use = 'Right'
    genome_seq_stretch = genome[v.CHROM][v.POS-lenalt:v.POS+lenalt]
    if (genome_seq_stretch == len(genome_seq_stretch) * genome_seq_stretch[0]):
        genome_seq_stretch_use = genome_seq_stretch
    else:
        genome_seq_stretch_use = 'Stretch'
    valt = v.ALT[0]
    vref = v.REF
    if len(v.ALT[0]) < len(v.REF):
        vref = vref.replace(valt, '', 1)
    elif len(v.ALT[0]) > len(v.REF):
        valt = valt.replace(vref, '', 1)
    else:
        pass

    if len(v.ALT[0]) > len(v.REF) and valt in genome_seq_stretch_use:
        #print('FAIL in stretch', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        failcount += 1 
    elif (len(v.ALT[0]) > len(v.REF)) and (valt not in genome_seq_stretch_use):
        #print('PASS not in stretch/stretch not homopolymer', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
    elif len(v.ALT[0]) < len(v.REF):
        if vref in valt:
            vref = vref.replace(valt, '', 1)
            if vref in genome_seq_left_use or genome_seq_right_use:
                #print('FAIL right or left', v.CHROM, v.POS, v.REF, v.ALT[0])
                #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                failcount += 1 
            else: 
                #print('PASS not in left or right', v.CHROM, v.POS, v.REF, v.ALT[0])
                #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
                passcount += 1
        else:
            #print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
            #print(genome_seq_left, genome_seq_right, genome_seq_stretch)    
            passcount += 1
    else:
        #print('PASS leftover', v.CHROM, v.POS, v.REF, v.ALT[0])
        #print(genome_seq_left, genome_seq_right, genome_seq_stretch)
        passcount += 1
print(v.INFO.get('donor') + ":", 'Passcount is', passcount, 'and failcount is', failcount) 
   


In [ ]:
def Substring(s):
 
    ans, temp = 1, 1
 
    # Traverse the string
    for i in range(1, len(s)):
         
        # If character is same as
        # previous increment temp value
        if (s[i] == s[i - 1]):
            temp += 1
        else:
            ans = max(ans, temp)
            temp = 1
 
    ans = max(ans, temp)
 
    # Return the required answer
    return ans
 
# Driver code
s = "abcdddddeff"
 
# Function call
print(Substring(s))
# Mohit Kumar 